In [0]:
import tensorflow as tf

In [0]:
# Define constants
image_dim = 28
num_labels = 10
batch_size = 50
num_steps = 1000
hidden_layers = [128, 32]

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!ls '/content/drive/My Drive/Tertiary Courses/Colab/DeepLearningTensorflow/mnist_records'

ls: cannot access '/content/drive/My Drive/Tertiary Courses/Colab/DeepLearningTensorflow/mnist_records': No such file or directory


In [0]:
data_path = '/content/drive/My Drive/Tertiary Courses/Colab/DeepLearningTensorflow/mnist_records'
train_path = data_path + '/mnist_train.tfrecords'
test_path = data_path + '/mnist_test.tfrecords'

In [0]:
# Step 1: Create a function to parse MNIST data
def parse_record(record):
    # Create a dictionary that describes the examples
    feature_dict = {'image_raw': tf.FixedLenFeature([], tf.string),
                    'label': tf.FixedLenFeature([], tf.int64)}
    
    parsed_record = tf.parse_single_example(record, feature_dict)
    
    image = tf.decode_raw(parsed_record['image_raw'], tf.int64)
    image.set_shape([image_dim * image_dim])
    image = tf.cast(image, tf.float32) / 255.
    
    label = tf.cast(parsed_record['label'], tf.int32)

    return image, label
    

In [11]:
# Step 2: Describe input data with a feature column
column = tf.feature_column.numeric_column('pixels', shape=[image_dim * image_dim])
column

NumericColumn(key='pixels', shape=(784,), default_value=None, dtype=tf.float32, normalizer_fn=None)

In [12]:
# Step 3: Create a DNNClassifier with the feature column
classifier = tf.estimator.DNNClassifier(
    feature_columns=[column],
    hidden_units=hidden_layers,
    n_classes=num_labels,
    optimizer=tf.train.AdamOptimizer(1e-4),
    dropout=0.1,
    model_dir='mnist')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'mnist', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff35ea94a20>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [16]:
# Step 4: Train the estimator
def train_func():
    dataset = tf.data.TFRecordDataset(train_path)
    dataset = dataset.map(parse_record).repeat().batch(batch_size)
    image, label = dataset.make_one_shot_iterator().get_next()
    return {'pixels': image}, label
classifier.train(train_func, steps=num_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from mnist/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1000 into mnist/model.ckpt.
INFO:tensorflow:loss = 26.37677, step = 1001
INFO:tensorflow:global_step/sec: 138.78
INFO:tensorflow:loss = 23.347652, step = 1101 (0.723 sec)
INFO:tensorflow:global_step/sec: 168.824
INFO:tensorflow:loss = 29.970013, step = 1201 (0.591 sec)
INFO:tensorflow:global_step/sec: 171.226
INFO:tensorflow:loss = 14.473545, step = 1301 (0.584 sec)
INFO:tensorflow:global_step/sec: 166.989
INFO:tensorflow:loss = 29.024515, step = 1401 (0.603 sec)
INFO:tensorflow:global_step/sec: 157.722
INFO:tensorflow:loss = 14.541679, step = 1501 (0.634 sec)
INFO:tensorflow:global_step/sec: 160.114
INFO:tensorflow:loss = 23.932283, step = 1601 (0.623

In [17]:
# Step 5: Test the estimator
def test_func():
    dataset = tf.data.TFRecordDataset(test_path)    
    dataset = dataset.map(parse_record).batch(batch_size)
    image, label = dataset.make_one_shot_iterator().get_next()
    return {'pixels': image}, label
metrics = classifier.evaluate(test_func)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-09-10T01:05:05Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from mnist/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-09-10-01:05:07
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.9226, average_loss = 0.28406075, global_step = 2000, loss = 14.203037
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: mnist/model.ckpt-2000


In [18]:
# Display metrics
for key, value in metrics.items():
    print(key, ': ', value)

accuracy :  0.9226
average_loss :  0.28406075
loss :  14.203037
global_step :  2000


In [0]:
# Step 1: Create a function to parse MNIST data
# def parser(record):
#     features = tf.parse_single_example(record,
#             features = {
#                     'images': tf.FixedLenFeature([], tf.string),
#                     'labels': tf.FixedLenFeature([], tf.int64),
#                     })
#     image = tf.decode_raw(features['images'], tf.uint8)
#     image.set_shape([image_dim * image_dim])
#     image = tf.cast(image, tf.float32) * (1.0/255) - 0.5
#     label = features['labels']
#     return image, label